In [0]:
from pyspark.sql.functions import col, to_date

# Load Bronze tables
df_bin_bronze = spark.table("bronze.bin_metadata")
df_logs_bronze = spark.table("bronze.collection_logs")

# Silver - bin_metadata
df_bin_silver = df_bin_bronze \
    .withColumn("latitude", col("latitude").cast("double")) \
    .withColumn("longitude", col("longitude").cast("double")) \
    .withColumn("capacity_liters", col("capacity_liters").cast("double")) \
    .withColumn("installation_date", to_date("installation_date", "yyyy-MM-dd"))

# Silver - collection_logs
df_logs_silver = df_logs_bronze \
    .withColumn("fill_level_percent", col("fill_level_percent").cast("double")) \
    .withColumn("latitude", col("latitude").cast("double")) \
    .withColumn("longitude", col("longitude").cast("double")) \
    .withColumn("date", to_date("date", "yyyy-MM-dd"))

# Save with schema overwrite
df_bin_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("silver.bin_metadata")

df_logs_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("silver.collection_logs")


In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze");# Load raw CSVs into DataFrames

df_bin_raw = spark.read.option("header", True).csv("/Volumes/project/project/project/bin_metadata.csv")
df_logs_raw = spark.read.option("header", True).csv("/Volumes/project/project/project/collection_logs.csv")

# Save as Bronze Delta Tables
df_bin_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.bin_metadata")
df_logs_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.collection_logs")
